In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [3]:
Multi_Class = pd.read_csv("C:\\Users\\graykar\\Downloads\\Pyhton\\Data Files\\dataset_30_page-blocks.csv")

In [4]:
Multi_Class.head()

,height,lenght,area,eccen,p_black,p_and,mean_tr,blackpix,blackand,wb_trans,class
0,5,7,35,1.400,0.400,0.657,2.33,14,23,6,1
1,6,7,42,1.167,0.429,0.881,3.60,18,37,5,1
2,6,18,108,3.000,0.287,0.741,4.43,31,80,7,1
3,5,7,35,1.400,0.371,0.743,4.33,13,26,3,1
4,6,3,18,0.500,0.500,0.944,2.25,9,17,4,1


In [5]:
Multi_Class['class'].value_counts()

1    4913
2     329
5     115
4      88
3      28
Name: class, dtype: int64

In [6]:
X_train, X_test, y_train, y_test = train_test_split( Multi_Class.iloc[: , 0:-1] , Multi_Class['class'] , test_size=0.20, random_state=1)

In [8]:
stdScaler = StandardScaler()
X_train_std = stdScaler.fit_transform(X_train)

In [9]:
X_train.head()

,height,lenght,area,eccen,p_black,p_and,mean_tr,blackpix,blackand,wb_trans
3817,28,32,896,1.143,0.501,0.744,7.02,449,667,64
5446,1,70,70,70.000,0.943,1.000,22.00,66,70,3
551,1,77,77,77.000,0.403,1.000,5.17,31,77,6
2584,8,7,56,0.875,0.375,1.000,2.63,21,56,8
4055,3,500,1500,166.670,0.373,0.421,19.28,559,632,29


In [10]:
sgd_clf = SGDClassifier(random_state=42, loss='log')
sgd_clf.fit(X_train_std, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [12]:
some_digit = X_train_std[1].reshape(1,-1)
some_digit_pred = sgd_clf.predict(some_digit)
some_digit_pred

array([2], dtype=int64)

In [13]:
sgd_clf.predict_proba(some_digit)

array([[7.74832886e-04, 9.99225167e-01, 3.31127751e-44, 1.09009012e-20,
        3.12332774e-42]])

In [14]:
sgd_clf.decision_function(some_digit)

array([[  -7.16752702,    5.07850297, -100.1218448 ,  -45.97087626,
         -95.57510966]])

In [15]:
np.argmax(sgd_clf.decision_function(some_digit))

1

In [16]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score,f1_score

In [17]:
y_train_pred = sgd_clf.predict(X_train_std)

In [18]:
conf_mx = confusion_matrix(y_train, y_train_pred)
conf_mx

array([[3914,   21,    0,    1,    2],
       [  62,  186,    0,    2,    3],
       [  10,    0,   10,    1,    0],
       [  14,    1,    0,   56,    0],
       [  68,    0,    1,    0,   26]], dtype=int64)

In [19]:
precision_score(y_train, y_train_pred, average='weighted')

0.9548188024198878

In [20]:
recall_score(y_train, y_train_pred, average='weighted')

0.9575148469620831

In [23]:
f1_score(y_train, y_train_pred, average='weighted')

0.9519495175012954

In [21]:
X_test_std = stdScaler.transform(X_test)
y_test_pred = sgd_clf.predict(X_test_std)
f1_score(y_test, y_test_pred, average='weighted')

0.9448459818074266

# One Vs All

In [24]:
from sklearn.multiclass import OneVsRestClassifier
onevsall = OneVsRestClassifier(SGDClassifier(random_state=32, loss='log'))
onevsall.fit(X_train_std, y_train)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=32, shuffle=True,
       tol=None, verbose=0, warm_start=False),
          n_jobs=1)

In [35]:
len(onevsall.estimators_)

5

In [30]:
onevsall.predict_proba(some_digit)

array([[7.28919356e-06, 9.99992711e-01, 5.09980283e-34, 1.17784495e-20,
        2.25737258e-39]])

In [31]:
onevsall.predict(some_digit)

array([2], dtype=int64)

# One Vs ONE

In [33]:
from sklearn.multiclass import OneVsOneClassifier
onevsone = OneVsOneClassifier(SGDClassifier(random_state=32, loss='log'))
onevsone.fit(X_train_std , y_train)
onevsone.predict(some_digit)

array([2], dtype=int64)

#### 5c2 combination of Estimator

In [34]:
len(onevsone.estimators_)

10

# Softmax Regression

In [36]:
from sklearn.linear_model import LogisticRegression
softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10, max_iter=1000, random_state=42)
softmax_reg.fit(X_train_std, y_train)
y_train_pred_sm = softmax_reg.predict(X_train_std)

In [38]:
confusion_matrix(y_train, y_train_pred_sm)

array([[3899,   29,    1,    2,    7],
       [  45,  205,    0,    2,    1],
       [   8,    0,   13,    0,    0],
       [   9,    1,    0,   61,    0],
       [  41,    0,    0,    0,   54]], dtype=int64)

In [39]:
precision_score(y_train, y_train_pred_sm, average='weighted')

0.9653316602983117

In [40]:
recall_score(y_train , y_train_pred_sm , average='weighted')

0.966651439013248

In [41]:
f1_score(y_train , y_train_pred_sm , average='weighted')

0.9650012020427585